https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

In [110]:
import pickle
with open("./names.pkl","rb") as file:
  names = pickle.load(file)

In [111]:
train_set = names[:int(len(names)*0.8)]
test_set = names[int(len(names)*0.8):]

In [112]:
languages_set = set()
for arr in [train_set,test_set]:
  for el in arr:
    languages_set.add(el[1])

In [113]:
languages_set=list(languages_set)

In [114]:
len(train_set),len(test_set), len(names)

(16059, 4015, 20074)

In [115]:
import string
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def letterToIndex(letter):
    return all_letters.find(letter)

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [116]:
import torch.nn as nn
import torch
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 256
rnn = RNN(n_letters, n_hidden, len(languages_set))

In [117]:
lr = 0.001 # If you set this too high, it might explode. If too low, it might not learn
from torch.optim import AdamW
optimizer = AdamW(rnn.parameters(), lr=lr)
criterion = nn.NLLLoss()
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    optimizer.zero_grad()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    return output, loss.item()

In [118]:
import time
import math
import random
n_iters = 200000
print_every = 5000
plot_every = 1000
from tqdm import tqdm
from statistics import mean

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def randomTrainingExample():
    el = random.choice(train_set)
    return lineToTensor(el[0]), torch.tensor([languages_set.index(el[1])], dtype=torch.long)

def test():
    losses = []
    with torch.no_grad():
      for el in test_set:
         line_tensor, category_tensor = lineToTensor(el[0]), torch.tensor([languages_set.index(el[1])], dtype=torch.long)
         hidden = rnn.initHidden()
         for i in range(line_tensor.size()[0]):
             output, hidden = rnn(line_tensor[i], hidden)
         loss = criterion(output, category_tensor)
         losses.append(loss.item())
    return mean(losses)


start = time.time()

for iter in tqdm(range(1, n_iters + 1)):
    line_tensor, category_tensor  = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        print("train_loss", all_losses[-1])
        print("test_loss",test())
        current_loss = 0

  0%|          | 993/200000 [00:03<11:25, 290.29it/s]

train_loss 1.831503066888501


  1%|          | 1053/200000 [00:06<1:05:51, 50.35it/s]

test_loss 1.6419061571904565


  1%|          | 1981/200000 [00:09<11:31, 286.49it/s]

...

train_loss 1.0176701570606004


 68%|██████▊   | 135053/200000 [13:52<22:57, 47.14it/s]

test_loss 1.0680651231391407


 68%|██████▊   | 135069/200000 [13:52<06:40, 162.22it/s]


KeyboardInterrupt: ignored

In [121]:
import numpy as np
languages_set2=np.array(languages_set)
with torch.no_grad():
    line_tensor = lineToTensor("Ivanov")
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    print(output)
    print(languages_set2[(-output).argsort()[0][:3]])

tensor([[-6.2351e-04, -1.1507e+01, -8.8284e+00, -1.1983e+01, -1.0631e+01,
         -1.6028e+01, -9.8823e+00, -1.1080e+01, -1.2941e+01, -1.2600e+01,
         -1.6891e+01, -1.4690e+01, -7.9562e+00, -1.2311e+01, -1.8311e+01,
         -1.2021e+01, -1.4164e+01, -1.3268e+01]])
['Russian' 'Arabic' 'Czech']
